In [11]:
"""
(C) Murray Shanahan et al, 2015
"""

import numpy as np


def NetworkRingLattice(N, k):
    """
    Creates a ring lattice with N nodes and neighbourhood size k.
    Choosing k = 2 connects each node to its nearest 2 nodes, k = 4
    to the nearest 4 and so on. Odd values of k are rounded down
    to the previous even number.
    You technically can choose k > N, but that will give you just a
    fully connected net.
    Inputs:
    N -- Number of nodes
    k -- Neighbourhood size of the initial ring lattice
    """

    # Create empty connectivity matrix
    CIJ = np.zeros([N, N])

    # Loop through the nodes and connect the neighbourhoods
    for i in range(N):
    # Note that since the network is undirected (symmetric) we only
    # need to look at the upper triangular bit of CIJ.
        for j in range(i+1, N):
            if i != j and min(abs(i-j), N - abs(i-j)) <= k/2.0:
                CIJ[i, j] = 1
                CIJ[j, i] = 1

    return(CIJ)

In [15]:
import numpy as np
import numpy.random as rn


def NetworkWattsStrogatz(N, k, p):
    """
    Creates a ring lattice with N nodes and neighbourhood size k, then
    rewires it according to the Watts-Strogatz procedure with probability p.
    Inputs:
    N -- Number of nodes
    k -- Neighbourhood size of the initial ring lattice
    p -- Rewiring probability
    """

    # Create a regular string lattice
    CIJ = NetworkRingLattice(N, k)

    # Loop over all connections and swap each of them with probability p
    for i in range(N):
        for j in range(i+1, N):
            if CIJ[i, j] and rn.random() < p:
                # We modify connections in both directions (i.e. [i,j] and [j,i])
                # to maintain network undirectedness (i.e. symmetry).
                CIJ[i, j] = 0
                CIJ[j, i] = 0
                # PEDRO
                # h = np.mod(i + np.ceil(rn.random()*(N-1)) - 1, N)
                h = int(np.mod(i + np.ceil(rn.random()*(N-1)) - 1, N))
                CIJ[i, h] = 1
                CIJ[h, i] = 1

    return(CIJ)

In [16]:
import numpy as np
from bct import breadthdist, charpath, clustering_coef_bu


def SmallWorldIndex(CIJ):
    """
    Computes the small-world index of the graph with connection matrix CIJ.
    Self-connections are ignored, as they are cyclic paths.
    Inputs:
    CIJ  --  Graph connectivity matrix. Must be binary (0 or 1) and undirected.
    """

    N = len(CIJ)
    K = np.sum(np.sum(CIJ))/len(CIJ)  # average degree

    # Clustering coefficient
    CC = np.mean(clustering_coef_bu(CIJ))

    # Distance matrix
    [RR, DD] = breadthdist(CIJ)

    # Note: the charpath implementation of bctpy is not very robust. Expect
    # some warnings. From the output of charpath use only the characteristic
    # path length
    PL = charpath(DD, include_diagonal=False)[0]

    # Calculate small-world index
    CCs = CC/(K/N)
    PLs = PL/(np.log(N)/np.log(K))
    SWI = CCs/PLs

    return(SWI)

In [ ]:
"""
Randomly generates networks according to the Watts-Strogatz procedure,
computes their small-world indices, and local and global efficiencies,
and plots them. N is the number of nodes, k is the neighbourhood size.
(C) Murray Shanahan et al, 2015
"""


import numpy as np
import matplotlib.pyplot as plt
import bct

# Set up parameter values
nb_trials = 100
N = 2000
k = 6

# Pre-allocate arrays
prob = 10**(-3*np.random.random(nb_trials))
SWI   = np.zeros(nb_trials)
Eglob = np.zeros(nb_trials)
Eloc  = np.zeros(nb_trials)

# Calculate statistics
for i, p in enumerate(prob):
    CIJ = NetworkWattsStrogatz(N, k, p)

    SWI[i]   = SmallWorldIndex(CIJ)
    Eglob[i] = bct.efficiency_wei(CIJ, local=False)
    Eloc[i]  = np.mean(bct.efficiency_wei(CIJ, local=True))

# Plot figures
plt.figure(1)
plt.semilogx(prob, SWI, marker='.', linestyle='none')
plt.xlabel('Rewiring probability')
plt.ylabel('Small World Index')
plt.grid(True)

plt.figure(2)
plt.subplot(211)
plt.semilogx(prob, Eglob, marker='.', linestyle='none')
plt.xlabel('Rewiring probability')
plt.ylabel('Global efficiency')
plt.grid(True)

plt.subplot(212)
plt.semilogx(prob, Eloc, marker='.', linestyle='none')
plt.xlabel('Rewiring probability')
plt.ylabel('Local efficiency')
plt.grid(True)

plt.show()